In [1]:
# ===========================================
# 금융상품 추천 시스템 개발(XGBoost)
# ===========================================

# 필요한 모든 라이브러리 import
import pandas as pd
import numpy as np
import json
import warnings
import joblib
warnings.filterwarnings('ignore')

# 머신러닝 라이브러리
from sklearn.model_selection import train_test_split
from sklearn.multioutput import MultiOutputClassifier
from sklearn.metrics import classification_report, precision_score, recall_score, f1_score, accuracy_score

# XGBoost
try:
    from xgboost import XGBClassifier
    import xgboost as xgb
    print("✅ 모든 라이브러리 import 성공!")
except ImportError as e:
    print(f"❌ Import 오류: {e}")
    print("XGBoost 설치: pip install xgboost")

print("🚀 XGBoost 금융상품 추천 시스템")
print("=" * 50)

✅ 모든 라이브러리 import 성공!
🚀 XGBoost 금융상품 추천 시스템


In [2]:
target_names = {
    'MMMF': '단기금융상품펀드',
    'CDS': '양도성예금증서', 
    'NMMF': '비머니마켓펀드',
    'STOCKS': '주식보유',
    'RETQLIQ': '퇴직준비금유동성'
}

In [4]:
# 데이터 로드 시도
try:
    df = pd.read_csv('data/SCFP/cleaned_scf_data.csv')
    X = df.iloc[:,:-5]  # 14개 독립변수
    y = df.iloc[:,-5:]  # 5개 타겟변수
    print(f"✅ 실제 데이터 로드: {X.shape[0]:,}명, {X.shape[1]}개 특성")
except FileNotFoundError:
    print("※ 실제 데이터 없음. 샘플 데이터 생성 중...")
    
    # 샘플 데이터 생성
    np.random.seed(42)
    n_samples = 2000
    X = pd.DataFrame(np.random.randn(n_samples, 16), 
                    columns=[f'feature_{i+1}' for i in range(16)])
    y = pd.DataFrame(np.random.randint(0, 2, (n_samples, 5)),
                    columns=list(target_names.keys()))
    print(f"✅ 샘플 데이터 생성: {X.shape[0]:,}명, {X.shape[1]}개 특성")

print(f"데이터 준비 완료: X={X.shape}, y={y.shape}")

✅ 실제 데이터 로드: 22,975명, 14개 특성
데이터 준비 완료: X=(22975, 14), y=(22975, 5)


In [5]:
# 데이터 분할
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

print(f"훈련 데이터: {X_train.shape}")
print(f"테스트 데이터: {X_test.shape}")
print("✅ 데이터 분할 완료")

# 타겟 불균형 확인
print("타겟 분포:")
print(y_train.sum() / len(y_train))

훈련 데이터: (18380, 14)
테스트 데이터: (4595, 14)
✅ 데이터 분할 완료
타겟 분포:
MMMF       0.041893
CDS        0.077476
NMMF       0.201251
STOCKS     0.290860
RETQLIQ    0.591023
dtype: float64


In [6]:
import numpy as np

# 특성 개수 확인 (colsample_bytree 계산용)
n_features = X.shape[1] 
colsample_ratio = np.sqrt(n_features) / n_features  # sqrt(16)/16 = 0.25

base_xgb_model = XGBClassifier(
    n_estimators=700,           
    max_depth=6,               
    learning_rate=0.035,       
    min_child_weight=6,        
    colsample_bytree=0.9,      
    subsample=0.9,             
    gamma=0.03,                
    reg_alpha=0.2,             
    reg_lambda=0.6,            
    verbosity=0,
    random_state=42
)

# class_weight='balanced' 처리를 위한 MultiOutputClassifier
multilabel_xgb = MultiOutputClassifier(base_xgb_model)

print("✅ XGBoost 모델 생성 완료")
print(f"📊 특성 개수: {n_features}, colsample_bytree: {colsample_ratio:.3f}")

✅ XGBoost 모델 생성 완료
📊 특성 개수: 14, colsample_bytree: 0.267


In [7]:
# multilabel_xgb 모델 훈련
print("🚀 MultiOutputClassifier 훈련 시작...")

# 데이터 확인
if 'X_train' in locals() and 'y_train' in locals():
    print(f"훈련 데이터: {X_train.shape}, 타겟: {y_train.shape}")
    
    # 훈련 실행
    multilabel_xgb.fit(X_train, y_train)
    print("✅ MultiOutputClassifier 훈련 완료!")
    
    # 테스트 예측
    test_pred = multilabel_xgb.predict(X_test)
    print(f"예측 결과 형태: {test_pred.shape}")
    
else:
    print("❌ 훈련 데이터가 없습니다.")
    print("먼저 데이터 로드 및 분할을 실행해주세요.")

🚀 MultiOutputClassifier 훈련 시작...
훈련 데이터: (18380, 14), 타겟: (18380, 5)
✅ MultiOutputClassifier 훈련 완료!
예측 결과 형태: (4595, 5)


In [8]:
# 테스트 데이터 예측
print("🔍 모델 예측 중...")
test_pred = multilabel_xgb.predict(X_test)
print("✅ 예측 완료")

🔍 모델 예측 중...
✅ 예측 완료


In [9]:
from sklearn.metrics import hamming_loss, jaccard_score

# 전체 성능 지표
accuracy = accuracy_score(y_test, test_pred)
precision = precision_score(y_test, test_pred, average='samples', zero_division=0)
recall = recall_score(y_test, test_pred, average='samples', zero_division=0)
f1 = f1_score(y_test, test_pred, average='samples', zero_division=0)

# Hamming Loss (더 현실적)
hamming = hamming_loss(y_test, test_pred)

# Jaccard Score (교집합/합집합)
jaccard = jaccard_score(y_test, test_pred, average='samples')

print("=" * 40)
print("★ XGBoost 전체 모델 성능")
print("=" * 40)
print(f"정확도 (Accuracy):  {accuracy:.3f}")
print(f"정밀도 (Precision): {precision:.3f}")
print(f"재현율 (Recall):    {recall:.3f}")
print(f"F1 점수:           {f1:.3f}")

print("=" * 40)
print("★ 평가 방식 추가")
print("=" * 40)
print(f"Hamming Loss: {hamming:.3f}")
print(f"Jaccard Score: {jaccard:.3f}")

★ XGBoost 전체 모델 성능
정확도 (Accuracy):  0.704
정밀도 (Precision): 0.570
재현율 (Recall):    0.529
F1 점수:           0.536
★ 평가 방식 추가
Hamming Loss: 0.072
Jaccard Score: 0.508


In [10]:
# 훈련 데이터 예측
train_pred = multilabel_xgb.predict(X_train)
train_accuracy = accuracy_score(y_train, train_pred)
test_accuracy = 0.706

print(f"훈련 정확도: {train_accuracy:.3f}")
print(f"테스트 정확도: {test_accuracy:.3f}")
print(f"차이: {train_accuracy - test_accuracy:.3f}")

훈련 정확도: 0.780
테스트 정확도: 0.706
차이: 0.074


In [11]:
# 교차검증으로 확인 필요
from sklearn.model_selection import cross_val_score
cv_scores = cross_val_score(multilabel_xgb, X_train, y_train, cv=5)
print(f"교차검증 점수: {cv_scores.mean():.3f}")

교차검증 점수: 0.688


In [12]:
# 각 금융상품별 정확도 확인
for i, target in enumerate(['MMMF', 'CDS', 'NMMF', 'STOCKS', 'RETQLIQ']):
    target_accuracy = accuracy_score(y_test.iloc[:, i], test_pred[:, i])
    print(f"{target}: {target_accuracy:.3f}")

MMMF: 0.974
CDS: 0.936
NMMF: 0.935
STOCKS: 0.899
RETQLIQ: 0.897


In [13]:
# "모든 것을 1로 예측" 했을 때 정확도
baseline_pred = np.ones_like(test_pred)
baseline_accuracy = accuracy_score(y_test, baseline_pred)
print(f"베이스라인: {baseline_accuracy:.3f}")

베이스라인: 0.002


In [14]:
# 훈련된 모델 저장
joblib.dump(multilabel_xgb, 'multilabel_xgb.joblib')
print("✅ 모델 저장 완료: multilabel_xgb.joblib")

✅ 모델 저장 완료: multilabel_xgb.joblib
